In [1]:
%matplotlib inline
import pandas
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np


pandas.set_option('display.max_columns', None)  
pandas.set_option('display.expand_frame_repr', False)
#pandas.set_option('max_colwidth', -1)
pandas.set_option('display.precision', 3)

In [3]:
data = pandas.read_csv('data.csv', sep=',', na_values="")
print(data[0:10])

       Date      Time                                         Tweet_Text  Type Media_Type                    Hashtags   Tweet_Id                                          Tweet_Url  twt_favourites_IS_THIS_LIKE_QUESTION_MARK  Retweets  Unnamed: 10  Unnamed: 11
0  16-11-11  15:26:37  Today we express our deepest gratitude to all ...  text      photo                   ThankAVet  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     127213     41112          NaN          NaN
1  16-11-11  13:33:35  Busy day planned in New York. Will soon be mak...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     141527     28654          NaN          NaN
2  16-11-11  11:14:20  Love the fact that the small groups of protest...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     183729 